# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame

Load the csv exported in Part I to a DataFrame

In [2]:
weather_df=pd.read_csv('../output/cities.csv')

weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Albany,42.6001,-73.9662,55.00,73,100,1.01,US,1619796182
1,Yellowknife,62.4560,-114.3525,27.00,93,90,14.97,CA,1619795602
2,Rikitea,-23.1203,-134.9692,78.46,75,58,18.63,PF,1619796385
3,Gombong,-7.6072,109.5142,55.00,100,73,2.98,ID,1619796385
4,Arraial do Cabo,-22.9661,-42.0278,77.00,69,40,9.22,BR,1619796386
...,...,...,...,...,...,...,...,...,...
523,Saint-Leu,-21.1500,55.2833,78.80,78,0,3.44,RE,1619796743
524,Dinguiraye,11.3000,-10.7167,98.01,22,96,4.41,GN,1619796744
525,Ulaanbaatar,47.9077,106.8832,32.00,74,40,4.47,MN,1619796744
526,Angoche,-16.2325,39.9086,80.20,77,9,5.70,MZ,1619796596


In [3]:
#Check types to prevent problems with data like in weatherpy.
print(weather_df.dtypes)

City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date            int64
dtype: object


In [4]:
#Change humidity
weather_df["Humidity"] = weather_df["Humidity"].astype(float)

### Humidity Heatmap

Configure gmaps.Use the Lat and Lng as locations and Humidity as the weight.Add Heatmap layer to map.

In [5]:
#latitude and longitude are locations
location = weather_df[["Lat", "Lng"]]

# Get humidity
hum_weight = weather_df["Humidity"]

In [6]:
#Choose type of google maps
fig = gmaps.figure(map_type="SATELLITE",center=(25, 0), zoom_level=1.7)

# Create the heat layer to go on top of map 
heat_layer = gmaps.heatmap_layer(location, weights=hum_weight, 
                                 dissipating=False,max_intensity=100,
                                 point_radius=2.6)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria

Narrow down the cities to fit weather conditions.Drop any rows will null values.

In [7]:
nice_weather = pd.DataFrame(weather_df[weather_df["Max Temp"].between(65, 85) & (weather_df['Wind Speed'] <= 15) & 
                                       (weather_df["Cloudiness"] <30)& (weather_df["Humidity"]<= 60)&(weather_df["Humidity"]>= 25)]) 
nice_weather.describe()

,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,28.000000,28.000000,28.000000,28.0000,28.000000,28.000000,2.800000e+01
mean,12.571593,-47.611118,72.306429,45.7500,4.535714,8.085000,1.619797e+09
std,31.606929,59.986122,4.946060,8.1678,8.774889,3.131965,2.031138e+02
min,-38.547300,-115.950000,65.970000,31.0000,0.000000,1.990000,1.619796e+09
25%,-23.173425,-89.535975,68.000000,39.7500,0.000000,5.930000,1.619796e+09
50%,30.141650,-58.860350,71.800000,44.5000,0.500000,8.165000,1.619797e+09
75%,37.185400,-12.064500,77.000000,52.2500,1.250000,10.360000,1.619797e+09
max,50.852300,139.500000,82.990000,60.0000,26.000000,14.970000,1.619797e+09


In [8]:
nice_weather.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,Mount Isa,-20.7333,139.5000,66.20,39.0,0,5.75,AU,1619796175
24,Castro,-24.7911,-50.0119,65.97,56.0,24,11.88,BR,1619795917
60,Três Passos,-27.4556,-53.9319,73.99,55.0,0,1.99,BR,1619796425
75,Moctezuma,29.8000,-109.6667,66.79,38.0,0,11.54,MX,1619796431
90,Avera,33.1940,-82.5271,77.00,57.0,1,11.50,US,1619796449


## Hotel Map
Store into variable named hotel_df.Add a "Hotel Name" column to the DataFrame.
Set parameters to search for hotels with 5000 meters.Hit the Google Places API for each city's coordinates.
Store the first Hotel result into the DataFrame.Plot markers on top of the heatmap.

In [9]:
#Create new column to store hotel data 
hotel_df= nice_weather


hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,Mount Isa,-20.7333,139.5000,66.20,39.0,0,5.75,AU,1619796175
24,Castro,-24.7911,-50.0119,65.97,56.0,24,11.88,BR,1619795917
60,Três Passos,-27.4556,-53.9319,73.99,55.0,0,1.99,BR,1619796425
75,Moctezuma,29.8000,-109.6667,66.79,38.0,0,11.54,MX,1619796431
90,Avera,33.1940,-82.5271,77.00,57.0,1,11.50,US,1619796449


In [10]:
#Add extra column
hotel_df["Hotel Name"] = " "
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
10,Mount Isa,-20.7333,139.5000,66.20,39.0,0,5.75,AU,1619796175,
24,Castro,-24.7911,-50.0119,65.97,56.0,24,11.88,BR,1619795917,
60,Três Passos,-27.4556,-53.9319,73.99,55.0,0,1.99,BR,1619796425,
75,Moctezuma,29.8000,-109.6667,66.79,38.0,0,11.54,MX,1619796431,
90,Avera,33.1940,-82.5271,77.00,57.0,1,11.50,US,1619796449,


In [11]:
#Select target type, distance and coordinates
target_type = "lodging"
coords = location
radius = 5000

# rewrite params dict

params = {
    "location": coords,
    "types": target_type,
    "radius": radius,
    "key": g_key
}

# Build URL using the Google Maps API
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

print("Searching for hotels near cities:")
print("-----------------------------------")

for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"


    # make request and print url
    hotel_response = requests.get(base_url, params=params)
    #Convert to json
    hotel_name = hotel_response.json()
    

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
        print(f'In {hotel_name["City"]}, you can find the{hotel_name["results"][0]["name"]}')
        #hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    
    except (KeyError, IndexError):
        print("Skipping.")

# print(response.url)

#hotel_name = hotel_response.json()
#print("Search has ended\n Enjoy your vacation")



Searching for hotels near cities:
-----------------------------------
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.


In [12]:
hotel_df.head(20)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
10,Mount Isa,-20.7333,139.5000,66.20,39.0,0,5.75,AU,1619796175,ibis Styles Mt Isa Verona
24,Castro,-24.7911,-50.0119,65.97,56.0,24,11.88,BR,1619795917,CHACARA BAILLY
60,Três Passos,-27.4556,-53.9319,73.99,55.0,0,1.99,BR,1619796425,Hotel Imperial
75,Moctezuma,29.8000,-109.6667,66.79,38.0,0,11.54,MX,1619796431,Hotel Lobohombo
90,Avera,33.1940,-82.5271,77.00,57.0,1,11.50,US,1619796449,
120,Caarapó,-22.6342,-54.8222,78.80,32.0,24,9.37,BR,1619796474,Hotel Pousada Quinta do Sul
146,Ponta do Sol,32.6667,-17.1000,66.20,52.0,20,10.36,PT,1619796496,Hotel do Campo
165,Rocha,-34.4833,-54.3333,68.13,54.0,0,8.28,UY,1619796429,Beleza Pura B&B
174,Necochea,-38.5473,-58.7368,72.00,52.0,0,5.99,AR,1619796520,dyd -mar Hotel
175,Huntington,38.4193,-82.4452,66.00,42.0,1,11.50,US,1619796520,Ramada Limited Huntington


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
     info_box_content=hotel_info)


# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))